<a href="https://colab.research.google.com/github/Migaalee/hadoop/blob/main/Copy_of_mapreduce2_final_exercise_IPs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Python MapReduce Exercise 2 _IPs

In the notebook, you should create a map-reduce program that count the number of occurrence of each word.

In this exercise, hadoop runs in standalone mode and reads data from the local filesystem.


### Download the dataset 

In [ ]:
!wget -O web.txt https://www.dropbox.com/s/0r8902uj9yum7dg/web.log?dl=0

--2020-10-12 15:42:47--  https://www.dropbox.com/s/0r8902uj9yum7dg/web.log?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.68.1, 2620:100:6024:1::a27d:4401
Connecting to www.dropbox.com (www.dropbox.com)|162.125.68.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/0r8902uj9yum7dg/web.log [following]
--2020-10-12 15:42:48--  https://www.dropbox.com/s/raw/0r8902uj9yum7dg/web.log
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uca38dbefe906d1a7119dd672e05.dl.dropboxusercontent.com/cd/0/inline/BBJWTh9SOzMwpSlMjSezBodf6RLKgEBwKUjB5o0rhlaRGCmZrHDdDMfjy2yGxK737qoGkYqrZotnkWwX7l0G5ohxzUzJQE4_sgW-Zzp7GXaqvg/file# [following]
--2020-10-12 15:42:49--  https://uca38dbefe906d1a7119dd672e05.dl.dropboxusercontent.com/cd/0/inline/BBJWTh9SOzMwpSlMjSezBodf6RLKgEBwKUjB5o0rhlaRGCmZrHDdDMfjy2yGxK737qoGkYqrZotnkWwX7l0G5ohxzUzJQE4_sgW-Zzp7GXaqvg/file
Resolving uca38dbefe906d1a

## Unique IPs Example
Read the file and calculate unique IP attacks.


### Mapper
Complete with the code for the mapper.

In [ ]:
%%file mapper_IP.py
#!/usr/bin/env python


# import sys
import sys
# import string library function  
import string  

# input comes from STDIN (standard input)
for line in sys.stdin:
    # remove leading and trailing whitespace
    line = line.strip()
    # remove punctuation characters
    # split the line into words
    words = line.split()
    for word in words:
        d=words[0]
        URL=words[1]
        print('%s\t%s' % (URL, d[:-10]))


Overwriting mapper_IP.py


### Reducer

In [ ]:
%%file reducer_IP.py
#!/usr/bin/python

import sys

# input comes from STDIN
for line in sys.stdin:
    # remove leading and trailing whitespace
    line = line.strip()

    # parse the input we got from mapper.py
    word, count = line.split('\t')

    unique_words=len(set(word))
        
print ("unique IPs are " + str(unique_words))

Writing reducer_IP.py


### Hadoop standalone mode execution


The output directory needs to be cleared...

In [ ]:
rm -rf results_IP

#### Submitting the job

The _hadoop_ command is used to submit the mapreduce job to the cluster...

In [ ]:
!hadoop jar /opt/hadoop-3.2.0/share/hadoop/tools/lib/hadoop-*streaming*.jar -files mapper_IP.py,reducer_IP.py -mapper mapper_IP.py -reducer reducer_IP.py -input web.txt -output results_IP

2020-10-12 18:24:58,802 INFO impl.MetricsConfig: Loaded properties from hadoop-metrics2.properties
2020-10-12 18:24:59,453 INFO impl.MetricsSystemImpl: Scheduled Metric snapshot period at 10 second(s).
2020-10-12 18:24:59,453 INFO impl.MetricsSystemImpl: JobTracker metrics system started
2020-10-12 18:24:59,561 WARN impl.MetricsSystemImpl: JobTracker metrics system already initialized!
2020-10-12 18:25:00,497 INFO mapred.FileInputFormat: Total input files to process : 1
2020-10-12 18:25:00,561 INFO mapreduce.JobSubmitter: number of splits:1
2020-10-12 18:25:01,088 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local106858373_0001
2020-10-12 18:25:01,088 INFO mapreduce.JobSubmitter: Executing with tokens: []
2020-10-12 18:25:01,720 INFO mapred.LocalDistributedCacheManager: Localized file:/home/jovyan/work/mapper_IP.py as file:/tmp/hadoop-jovyan/mapred/local/job_local106858373_0001_dc861a46-1ef6-4afc-864c-e57772eb456e/mapper_IP.py
2020-10-12 18:25:01,754 INFO mapred.LocalDis

#### Checking the results
The result is stored in directory results.

In [ ]:
!cat results_IP/part-*

unique IPs are 7	


## Now calculate average attack time, min, max for every 10 sec


### Mapper
Complete with the code for the mapper.

In [ ]:
%%file mapper_time.py
#!/usr/bin/env python

# import sys
import sys
# import string library function  
import string  

# input comes from STDIN (standard input)
for line in sys.stdin:
    # remove leading and trailing whitespace
    line = line.strip()
    # remove punctuation characters
    # split the line into words
    words = line.split()
    for word in words:
        d=words[0]
        attack=words[5]
        print('%s\t%s' % (d[:-10], attack[:7]))

Overwriting mapper_time.py


### Reducer

In [ ]:
%%file reducer_time.py
#!/usr/bin/python

import sys
import string


interval_time={} #made a dictionary


# input comes from STDIN
for line in sys.stdin:
    # remove leading and trailing whitespace
    line = line.strip()

    # parse the input we got from mapper.py
    interval, time = line.split('\t')
    
    # convert time (currently a string) to float
    
    time=float(time)
    
    time=time*1000

    
    time=int(time)
    
    
    if interval in interval_time:
        interval_time[interval].append(int(time))
        
    else:
        interval_time[interval] = []
        interval_time[interval].append(int(time))

#Reducer
for interval in interval_time.keys():
    
    ave_time = sum(interval_time[interval])*1.0 / len(interval_time[interval])
    
    ave_time = str(ave_time)
    
    num_requests = len(interval_time[interval])
    
    min_time = min(interval_time[interval])
    
    max_time = max(interval_time[interval])
    
    print('%s\t%s\t%s\t%s\t%s' % (interval, ave_time[:5], num_requests, min_time, max_time))
    
  
    
    
    
    
    
    
    


Overwriting reducer_time.py


### Hadoop standalone mode execution


The output directory needs to be cleared...

In [ ]:
rm -rf results_time

#### Submitting the job

The _hadoop_ command is used to submit the mapreduce job to the cluster...

In [ ]:
!hadoop jar /opt/hadoop-3.2.0/share/hadoop/tools/lib/hadoop-*streaming*.jar -files mapper_time.py,reducer_time.py -mapper mapper_time.py -reducer reducer_time.py -input web.txt -output results_time

2020-10-13 08:23:17,688 INFO impl.MetricsConfig: Loaded properties from hadoop-metrics2.properties
2020-10-13 08:23:17,930 INFO impl.MetricsSystemImpl: Scheduled Metric snapshot period at 10 second(s).
2020-10-13 08:23:17,930 INFO impl.MetricsSystemImpl: JobTracker metrics system started
2020-10-13 08:23:17,962 WARN impl.MetricsSystemImpl: JobTracker metrics system already initialized!
2020-10-13 08:23:18,142 INFO mapred.FileInputFormat: Total input files to process : 1
2020-10-13 08:23:18,165 INFO mapreduce.JobSubmitter: number of splits:1
2020-10-13 08:23:18,383 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local738032066_0001
2020-10-13 08:23:18,383 INFO mapreduce.JobSubmitter: Executing with tokens: []
2020-10-13 08:23:18,764 INFO mapred.LocalDistributedCacheManager: Localized file:/home/jovyan/work/mapper_time.py as file:/tmp/hadoop-jovyan/mapred/local/job_local738032066_0001_08aa4a4d-e6b9-4e18-8de8-ff3f60debd27/mapper_time.py
2020-10-13 08:23:18,786 INFO mapred.Loca

#### Checking the results
The result is stored in directory results.

In [ ]:
!cat results_time/part-*

2016-12-06T10:01:4	123.7	408	117	166
2016-12-06T10:02:5	123.4	366	117	165
2016-12-06T10:01:5	121.9	228	117	141
2016-12-06T09:55:3	8.0	6	8	8
2016-12-06T10:02:0	125.4	282	118	290
2016-12-06T10:02:4	124.3	300	117	184
2016-12-06T09:00:5	200.0	48372	6	1905
2016-12-06T09:00:4	325.7	72810	7	9537
2016-12-06T09:00:1	7857.	58314	225	34406
2016-12-06T09:00:0	8649.	41292	17	45314
2016-12-06T09:00:3	1604.	40626	7	26530
2016-12-06T09:00:2	4611.	39696	14	25847
2016-12-06T09:01:0	175.4	52482	5	1624
2016-12-06T09:01:1	148.0	63312	4	1506
2016-12-06T09:01:2	153.6	31890	5	1361
2016-12-06T09:01:3	116.5	36978	5	1117
2016-12-06T09:01:4	100.1	31764	5	1339
2016-12-06T09:01:5	98.41	20058	5	1098
2016-12-06T09:04:1	67.16	72	59	143
2016-12-06T09:04:0	60.0	18	60	60
2016-12-06T09:04:2	60.16	36	59	62
2016-12-06T09:05:0	61.77	54	59	65
2016-12-06T09:04:4	60.5	36	59	64
2016-12-06T10:02:3	126.6	372	117	280
2016-12-06T10:02:2	123.2	498	118	191
2016-12-06T10:03:0	122.4	294	117	137
2016-12-06T10:02:1	122.1	324	117	133
2016-

## Create an inverted index that for each interval of 10s, has a list of unique IPs


### Mapper (although could re-use mapper.IP.py)
Complete with the code for the mapper.

In [ ]:
%%file mapper_invert.py
#!/usr/bin/env python


# import sys
import sys
# import string library function  
import string  

# input comes from STDIN (standard input)
for line in sys.stdin:
    # remove leading and trailing whitespace
    line = line.strip()
    # remove punctuation characters
    # split the line into words
    words = line.split()
    for word in words:
        d=words[0]
        URL=words[1]
        print('%s\t%s' % (d[:-10], URL))


Writing mapper_invert.py


### Reducer

In [ ]:
%%file reducer_invert.py
#!/usr/bin/python

import sys

interval_URL={} #made a dictionary

# input comes from STDIN
for line in sys.stdin:
    # remove leading and trailing whitespace
    line = line.strip()

    # parse the input we got from mapper.py
    interval, URL = line.split('\t')
    
    
    if interval in interval_URL:
        interval_URL[interval].append(URL)
        
    else:
        interval_URL[interval] = []
        interval_URL[interval].append(URL)

#Reducer

for interval in interval_URL.keys():
    
    unique_URL = set(interval_URL[interval])
    
    
    print('%s\t%s' % (interval, unique_URL))
    

Overwriting reducer_invert.py


In [ ]:
rm -rf results_invert

In [ ]:
!hadoop jar /opt/hadoop-3.2.0/share/hadoop/tools/lib/hadoop-*streaming*.jar -files mapper_invert.py,reducer_invert.py -mapper mapper_invert.py -reducer reducer_invert.py -input web.txt -output results_invert

2020-10-13 09:00:10,429 INFO impl.MetricsConfig: Loaded properties from hadoop-metrics2.properties
2020-10-13 09:00:10,534 INFO impl.MetricsSystemImpl: Scheduled Metric snapshot period at 10 second(s).
2020-10-13 09:00:10,534 INFO impl.MetricsSystemImpl: JobTracker metrics system started
2020-10-13 09:00:10,556 WARN impl.MetricsSystemImpl: JobTracker metrics system already initialized!
2020-10-13 09:00:10,728 INFO mapred.FileInputFormat: Total input files to process : 1
2020-10-13 09:00:10,749 INFO mapreduce.JobSubmitter: number of splits:1
2020-10-13 09:00:10,944 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local1189265606_0001
2020-10-13 09:00:10,944 INFO mapreduce.JobSubmitter: Executing with tokens: []
2020-10-13 09:00:11,274 INFO mapred.LocalDistributedCacheManager: Localized file:/home/jovyan/work/mapper_invert.py as file:/tmp/hadoop-jovyan/mapred/local/job_local1189265606_0001_b7c32206-6cdc-401f-bcb5-d97e42b968fc/mapper_invert.py
2020-10-13 09:00:11,294 INFO mapre

In [ ]:
!cat results_invert/part-*

2016-12-06T10:01:4	set(['106.37.189.69', '123.127.217.155'])
2016-12-06T10:02:5	set(['222.35.13.232', '106.37.189.69', '123.127.217.155'])
2016-12-06T10:01:5	set(['106.37.189.69', '123.127.217.155'])
2016-12-06T09:55:3	set(['88.157.128.134'])
2016-12-06T10:02:0	set(['106.37.189.69', '123.127.217.155'])
2016-12-06T10:02:4	set(['222.35.13.232', '106.37.189.69', '123.127.217.155'])
2016-12-06T09:00:5	set(['2602:ff62:104:7c9:8000::', '114.215.150.13', '178.22.148.122', '71.183.112.122', '123.207.96.93', '182.35.20.128', '101.200.154.90', '103.1.236.112', '61.132.241.109', '97.77.104.22', '200.168.250.196', '2002:894a:3a93:d:250:56ff:fe00:88c0', '217.61.2.106', '200.7.160.61', '202.47.236.252', '211.140.26.58', '46.14.171.138', '181.215.103.234', '109.238.227.27', '2001:41d0:a:2417::1', '125.31.1.27', '202.106.16.36', '185.15.43.51', '2a01:488:66:1000:5c33:8503:0:1', '192.241.151.220', '41.76.44.76', '120.25.166.134', '41.169.97.106', '177.54.250.18', '37.139.9.11', '201.18.115.114', '2001: